# COMP 551 — Assignment 4

Authors:
 - Bernier, Audréanne
 - Coull-Neveu, Ben
 - Trachsel-Bourbeau, Anjara

Imports

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
%matplotlib inline

import torch.nn as nn

import os
import pickle # to save models

Plot defaults

In [3]:
# define plotting parameters
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 14
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['legend.fancybox'] = False
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.autolimit_mode'] = 'data'  # default, ensures autoscale uses data
plt.rcParams["font.family"] = "serif"


# set default save directory and parameters
SAVEDIR = './figures/'
MODELDIR = './models/'
os.makedirs(SAVEDIR, exist_ok=True)
os.makedirs(MODELDIR, exist_ok=True)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

# Get data

X is input data that include text sequences

Y is target value 

YL1 is target value of level one (parent label) --> scientific field {0-6}

YL2 is target value of level one (child label) --> subfield {0-32}

In [4]:
def load_data(file_path, dtype=str)->np.ndarray:
    with open(file_path, 'r') as f:
        data = []
        line = f.readlines()
        while line != '':
            data.append(line)
            line = f.readline()
    return np.array(data, dtype=dtype).squeeze()



# loading WOS11967
dataset_dir = './datasets/WOS11967/'

input_file = dataset_dir + 'X.txt'
X = load_data(input_file)
    
target_file = dataset_dir + 'Y.txt'
Y = load_data(target_file, dtype=int)

targetL1_file = dataset_dir + 'YL1.txt'
YL1 = load_data(targetL1_file, dtype=int)

targetL2_file = dataset_dir + 'YL2.txt'
YL2 = load_data(targetL2_file, dtype=int)

Labels in YL1 (scientific field):

- 0 - Computer Science
- 1 - Electrical Engineering
- 2 - Psychology
- 3 - Mechanical Engineering
- 4 - Civil Engineering
- 5 - Medical Science
- 6 - Biochemistry


# Implement LSTM class

- can use pytorch.nn.Module
- For the LSTM model, you must design the data pre-processing pipeline that turns the unstructured text data into
numerical features. You are free to choose your encoding method, including pre-trained methods like word2vec;
however, there should be some justification for your choice in your report.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x):
        pass
    
    def fit(self, X, Y):
        pass
    
    def predict(self, X):
        pass
    
    def evaluate_acc(self, Y, Yhat):
        return np.mean(Y == Yhat)

# Implement BERT

- For BERT, you can use the transformers package to tokenize the input text and convert the tokens into numeri-
cal features https://pytorch.org/hub/huggingface_pytorch-transformers/
- 